In [ ]:
# %load_ext autoreload
# %autoreload 2

In [ ]:
import os 
import glob
import numpy as np
import matplotlib.pyplot as plt

from PIL import Image
from torchvision import transforms


try:
    from torchvision.transforms import InterpolationMode

    BICUBIC = InterpolationMode.BICUBIC
except ImportError:
    BICUBIC = Image.BICUBIC
    

In [ ]:
def to_numpy(tensor):
    return (
        tensor.detach().cpu().numpy() if tensor.requires_grad else tensor.cpu().numpy()
    )


def _convert_image_to_rgb(image):
    return image.convert("RGB")


def preprocess_img(img):
    transform = transforms.Compose(
        [
            transforms.Resize(448, BICUBIC),
            transforms.CenterCrop(448),
            _convert_image_to_rgb,
        ]
    )
    return transform(img)

In [ ]:
scribbles_folder = '../datasets/SCRIBBLES'
scribbles_paths = sorted(glob.glob(scribbles_folder + "/*.png"))[::-1][:1000] # For heavy masking [::-1] 
scribbles_paths[:5]
# https://github.com/hasibzunair/masksup-segmentation/blob/master/notebooks/exp_dataloader.ipynb

In [ ]:
scribble = Image.open(scribbles_paths[10]).convert('P')
scribble

In [ ]:
scribble_pre = preprocess_img(scribble)
scribble_pre

In [ ]:
scribble_pre.size